# Data generating
1. generate data 
2. load data

In [185]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split

torch.manual_seed(1)    # reproducible

data_size = 20 # n = 10000000 the minimum is 81, so hard to find 64 which is the global minimum
x = torch.randint(1,10,(data_size,8))
x = x.float()
y = torch.sum(x,1)**2
print(x)
print(y.t())

full_dataset = torch.cat([x, y], dim=0)
print(full_dataset)

# train_size = int(0.8 * len(data_size))
# test_size = len(data_size) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])




# print(x)
# print(y)
# print(min(y))


tensor([[5., 6., 1., 6., 8., 2., 3., 6.],
        [9., 1., 3., 4., 2., 9., 5., 1.],
        [4., 7., 3., 8., 7., 7., 9., 8.],
        [7., 1., 8., 9., 9., 5., 6., 3.],
        [7., 7., 8., 7., 9., 7., 3., 3.],
        [7., 6., 6., 6., 1., 7., 1., 7.],
        [3., 5., 4., 4., 5., 7., 1., 8.],
        [6., 9., 4., 4., 5., 8., 4., 6.],
        [4., 4., 8., 6., 2., 8., 5., 3.],
        [3., 9., 8., 4., 3., 4., 9., 8.],
        [4., 1., 9., 1., 2., 7., 8., 5.],
        [8., 7., 6., 5., 2., 7., 1., 1.],
        [1., 3., 3., 8., 6., 6., 7., 8.],
        [5., 7., 2., 6., 7., 8., 2., 5.],
        [9., 7., 2., 9., 7., 2., 6., 4.],
        [5., 8., 3., 3., 3., 3., 8., 6.],
        [2., 4., 7., 9., 4., 4., 2., 6.],
        [4., 1., 7., 1., 7., 5., 6., 6.],
        [1., 9., 1., 3., 4., 8., 1., 9.],
        [7., 7., 5., 1., 8., 1., 8., 2.]])


RuntimeError: t() expects a 2D tensor, but self is 1D

# Classification of datasets into good and bad value

In [ ]:
y_sort = sorted(y) # note that sorted change the sort but didn't change the original one
# print(results_map_sort)
median_value = y_sort[data_size//2]
# median_value = 3.5
# print(median_value)

# y_new = torch.LongTensor(data_size,1)
y_new = torch.zeros(data_size,1)

for i in range(data_size):
    if y[i] > median_value:
        y_new[i] = 1
    else:
        y_new[i] = 0

# y_new = y_new.long()
print(y_new)

# Create NN and optimizer

In [177]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.out = torch.nn.Linear(n_hidden, n_output)   # output layer
        self.out_act = torch.nn.Sigmoid()
#         self.out_act = torch.nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.out(x)
        x = self.out_act(x)
        return x

net = Net(n_feature=8, n_hidden=100, n_output=1)     # define the network
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.02)
# loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted
loss_func = torch.nn.BCELoss()

out = net(x)                 # input x and predict based on x
loss = loss_func(out, y_new) 
print(out)
# print(loss)
    

Net(
  (hidden): Linear(in_features=8, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
  (out_act): Sigmoid()
)
tensor([[0.4322],
        [0.6143],
        [0.4486],
        [0.5717],
        [0.3903],
        [0.4116],
        [0.3882],
        [0.3752],
        [0.4354],
        [0.4010],
        [0.4029],
        [0.3624],
        [0.5032],
        [0.4551],
        [0.4941],
        [0.4040],
        [0.5068],
        [0.4126],
        [0.3925],
        [0.3033],
        [0.3859],
        [0.4518],
        [0.4153],
        [0.5760],
        [0.3988],
        [0.5258],
        [0.4778],
        [0.4342],
        [0.3939],
        [0.4165],
        [0.5835],
        [0.4182],
        [0.4061],
        [0.6304],
        [0.5006],
        [0.3618],
        [0.5770],
        [0.5068],
        [0.5334],
        [0.4223],
        [0.3138],
        [0.4502],
        [0.4741],
        [0.4111],
        [0.3346],
        [0.4407],
        [0.4742],
 

# Start Training

In [ ]:



def train_epoch(model, opt, criterion, batch_size=50):
    model.train()
    losses = []
    for beg_i in range(0, X.size(0), batch_size):
        x_batch = X[beg_i:beg_i + batch_size, :]
        y_batch = Y[beg_i:beg_i + batch_size, :]
        x_batch = Variable(x_batch)
        y_batch = Variable(y_batch)

        opt.zero_grad()
        # (1) Forward
        y_hat = net(x_batch)
        # (2) Compute diff
        loss = criterion(y_hat, y_batch)
        # (3) Compute gradients
        loss.backward()
        # (4) update weights
        opt.step()        
        losses.append(loss.data.numpy())
    return losses

e_losses = []
num_epochs = 20
for e in range(num_epochs):
    e_losses += train_epoch(net, opt, criterion)
plt.plot(e_losses)

# plt.ion()   # something about plotting

# for t in range(100):
#     out = net(x)                 # input x and predict based on x
#     loss = loss_func(out, y_new)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

#     optimizer.zero_grad()   # clear gradients for next train
#     loss.backward()         # backpropagation, compute gradients
#     optimizer.step()        # apply gradients
    
# #     print(loss.item())
#     print(loss.data.numpy())

# #     if t % 2 == 0:
# #         # plot and show learning process
# #         plt.cla()
# #         prediction = torch.max(out, 1)[1]
# #         pred_y = prediction.data.numpy()
# #         target_y = y.data.numpy()
# #         plt.scatter(x.data.numpy()[:, 0], x.data.numpy()[:, 1], c=pred_y, s=100, lw=0, cmap='RdYlGn')
# #         accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
# #         plt.text(1.5, -4, 'Accuracy=%.2f' % accuracy, fontdict={'size': 20, 'color':  'red'})
# #         plt.pause(0.1)

# # plt.ioff()
# # plt.show()

In [ ]:
out = net(x) 
print(out)